In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoProcessor, pipeline, AutoModelForCausalLM
import csv
import torch


In [5]:
#Load the model from local only
model = AutoModelForCausalLM.from_pretrained("llama", torch_dtype=torch.bfloat16, device_map="auto", local_files_only=True)
# Load the tokenizer and processor from local
tokenizer = AutoTokenizer.from_pretrained("llama", local_files_only=True)
processor = AutoProcessor.from_pretrained("llama", local_files_only=True)




/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
df = pd.read_csv('training_data_segmented.csv')
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,GOID,Title,Text,Date,Class,segment_25,context,segment_50
0,0,0,411,1820265981,"January 2, 1972 (Page 2 of 178)","2-A Sunday, Jan. 2, '72 DETROIT FREE PRESS 197...",1972-01-02,1,"['; Gus Smith, 21, shot by police In the attem...","['; Gus Smith, 21, shot by police In the attem...","[' shot in the line of duty; Gus Smith, 21, sh..."
1,1,1,4088,1813612520,"May 16, 1925 (Page 10 of 22)",12 THE DETROIT FR E PRESS. SAT l J.JA . m a i ...,1925-05-16,0,"['and of gunmen, wbleh the police said numbere...","['and of gunmen, wbleh the police said numbere...","['lk. valued at Ono, by a band of gunmen, wble..."
2,2,2,9596,1821803197,"June 21, 1979 (Page 12 of 72)","1A ctwir vr.it F'Kc:3TNUP?DAy, ju?.t :i, 1973 ...",1979-06-21,0,"['LPHIA - (AP) Hundreds of police, aided by st...","['LPHIA - (AP) Hundreds of police, aided by st...",[' sniper kills boy PHILADELPHIA - (AP) Hundre...
3,3,3,11521,1820095901,"September 22, 1971 (Page 3 of 52)",Free Press Telephones it jfaee Today's Chuckle...,1971-09-22,1,"[""dent of Michigan's black police association,...","[""dent of Michigan's black police association,...",['ii Staff Writer The president of Michigan\'s...
4,4,4,13087,1822204708,"July 4, 1982 (Page 41 of 202)","DETROIT FREE PRESSSUNDAY, JULY 4, 1982 11D Riv...",1982-07-04,0,"['n, maintenance, military police and medical ...","['n, maintenance, military police and medical ...","[' in signal, transportation, maintenance, mil..."


In [7]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def correct_ocr_with_prompt(texts, batch_size=8):
    corrected_texts = []
    for i in range(0, len(texts), batch_size):
        batch_text = texts[i:i + batch_size]
        prompt_batch = [f"Correct the OCR errors in the following text:\n{text}\nCorrected text:" for text in batch_text]

        # Ensure padding is applied
        inputs = tokenizer(
            prompt_batch, 
            return_tensors='pt', 
            padding=True, 
            truncation=True
        )

        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_length= 600,
                num_return_sequences=1,
                temperature=0.7,
                top_k=50,
                top_p=0.92
            )

        decoded_texts = tokenizer.batch_decode(output, skip_special_tokens=True)
        corrected_batch = [decoded.split("Corrected text:")[-1].strip() for decoded in decoded_texts]
        corrected_texts.extend(corrected_batch)

    return corrected_texts


# Apply the batch correction function to the column
df['corrected_text'] = correct_ocr_with_prompt(df['segment_25'].tolist())

# Display the resulting DataFrame
print(df[['segment_25', 'corrected_text']])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_tok

                                            segment_25  \
0    ['; Gus Smith, 21, shot by police In the attem...   
1    ['and of gunmen, wbleh the police said numbere...   
2    ['LPHIA - (AP) Hundreds of police, aided by st...   
3    ["dent of Michigan's black police association,...   
4    ['n, maintenance, military police and medical ...   
..                                                 ...   
136  ["s how women of Detroit's police department w...   
137  ['elligence Section of the police department, ...   
138  [" for a new firemen's and police-men's police...   
139  ['arly intervention by the police and by the J...   
140  ['nger forefinger at him," police said Kulas t...   

                                        corrected_text  
0    1. Gus Smith, 21, shot by police in the attemp...  
1    So, the police said, the gunmen numbered betwe...  
2    3 'LPHIA - (AP) Hundreds of police, aided by s...  
3    by members of the police department's controve...  
4    ['n, maintena

In [9]:
df.to_csv("post_correction.csv", index=False)